In [1]:
from utils import *
from utils2 import *
from gpcrdb_soup import *

In [2]:
import Bio
import yaml
import os
from functools import partial
from operator import is_not
from tqdm import tqdm

In [3]:
def disp3(df: pd.DataFrame):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df.head(3))

In [4]:
uniprot = False  # specify if we want to scrape for uniprot links ~

In [5]:
# table = get_table(reload=True, save=True, uniprot=uniprot)

In [6]:
table = get_table(reload=False)

In [7]:
if uniprot:
    t = table[['PDB', 'pdb_link', 'uniprot(gene)','uniprot_id', 'uniprot_link']]
else:
    t = table[['PDB', 'pdb_link', 'uniprot(gene)']]

In [118]:
# t
# Todo: https://www.ebi.ac.uk/training/online/sites/ebi.ac.uk.training.online/files/UniProt_programmatically_py3.pdf

In [9]:
df = table

# This is the thing we might want to specify with argparse
filtered = df[df['Cl.'].str.contains('A')]
filtered = filtered[filtered['Species'].str.contains('Human')]

In [117]:
# filtered

In [11]:
# 2.1) find active state (complex i.p.)
filtered_complex = filtered[filtered['Family'].str.contains('Gs')]
active = filtered_complex[filtered_complex['State'].str.contains('Active')]

In [119]:
# disp3(active)

In [13]:
# todo: check if file already exists and use overwrite=True/False
"""
for url in tqdm(active['pdb_link']):
    download_pdb(url, 'active')
"""

"\nfor url in tqdm(active['pdb_link']):\n    download_pdb(url, 'active')\n"

In [14]:
# Data loading
files_a, prots_a = get_pdb_files(path='data/pdb/active/')  # Get all downloaded pdb files in specified path
print("Found {} proteins: {}.".format(len(prots_a), prots_a))

Found 13 proteins: ['7CFN', '5G53', '7CFM', '6NI3', '6GDG', '6E67', '7BZ2', '7DHI', '7DHR', '7D7M', '3SN6', '7BW0', '6LI3'].


In [15]:
# find inactive counter parts of the active proteins ==> use gene/uniprot, Family and species to match them

genes = list(set(active['uniprot(gene)'].values.tolist()))
print(genes)

inactive = filtered[(filtered['uniprot(gene)'].isin(genes))
                    & (filtered['State'].str.contains('Inactive')) 
                    & (filtered['Species'].str.contains('Human'))]

['ADRB2', 'AA2AR', 'GPBAR', 'GPR52', 'PE2R4']


In [16]:
"""for url in tqdm(inactive['pdb_link']):
    download_pdb(url, 'inactive')"""

"for url in tqdm(inactive['pdb_link']):\n    download_pdb(url, 'inactive')"

In [120]:
# disp3(inactive)

In [18]:
# Data loading
files_i, prots_i = get_pdb_files(path='data/pdb/inactive/')  # Get all downloaded pdb files in specified path
print("Found {} proteins: {}.".format(len(prots_i), prots_i))

Found 74 proteins: ['5OLG', '3VG9', '3D4S', '3PWH', '6ZDR', '2RH1', '6LPL', '5IU7', '6AQF', '6PS5', '3NY8', '3NY9', '5JQH', '5OLO', '6PS6', '6PS7', '5UVI', '5K2D', '5OM1', '6PS1', '6GT3', '5YWY', '5D5B', '6PS4', '5NM4', '3KJ6', '3VGA', '5OLH', '3PDS', '5IUB', '6S0Q', '5D5A', '6PS2', '5UIG', '6PRZ', '5IU4', '5OLV', '6PS3', '2R4S', '5K2B', '4GBR', '5N2R', '5JTB', '6OBA', '6WQA', '6LPK', '3NYA', '5NLX', '5VRA', '6PS0', '5MZJ', '5YHL', '6S0L', '4EIY', '5D6L', '5K2C', '5OM4', '6JZH', '6MH8', '5NM2', '2R4R', '3EML', '5OLZ', '6ZDV', '3REY', '3UZC', '5IU8', '5K2A', '3RFM', '5IUA', '5MZP', '5X7D', '3UZA', '6LPJ'].


In [19]:
try:
    seg_aligns = get_alignment(prots_a)
    print(yaml.dump(seg_aligns, default_flow_style=False))
except:
    print("Doesnt work anymore...")

Searching for alignment: https://gpcrdb.org/services/alignment/protein/7CFN,5G53,7CFM,6NI3,6GDG,6E67,7BZ2,7DHI,7DHR,7D7M,3SN6,7BW0,6LI3/TM7/
Doesnt work anymore...


In [20]:
# Solution: manual download for the entire A class

In [21]:
al_df = clean_alignment(load_alignment_())

In [22]:
len(al_df)  # this is the entire class A!

598

In [121]:
# disp3(al_df)

In [24]:
# since the identifiers are only human readable...

In [25]:
genes = list(al_df['clean_id'])

l4=[]
l5=[]
l6=[]

for g in genes:
    if len(g) == 4:
        l4.append(g)
    elif len(g) == 5:
        l5.append(g)
    elif len(g) == 6:
        l6.append(g)
    else:
        pass

In [26]:
# print(l4)  # these are (all?) pdbs

In [27]:
# print(l5)  # these are?

In [28]:
# print(l6)  # another id..

In [29]:
# This should be put into a function somewhere or just left out!
inactive['pdb_lower'] = inactive.PDB.apply(lambda x: x.lower())
active['pdb_lower'] = active.PDB.apply(lambda x: x.lower())

<ipython-input-29-29d6b45c2935>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inactive['pdb_lower'] = inactive.PDB.apply(lambda x: x.lower())


In [30]:
found = []
for g in l4:
    select = inactive[inactive['pdb_lower'].str.contains(g.lower())]
    if len(select) > 0:
        found.append(g)

In [31]:
print("Found {} / {} inactive proteins.".format(len(found), len(inactive)))

Found 74 / 74 inactive proteins.


In [32]:
# check
# print(sorted(list(inactive['PDB'].str.lower()))[:10])
# print(sorted(l4)[:10])

In [33]:
structures_a = pdb_data(files_a)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  5.02it/s]


In [34]:
structures_i = pdb_data(files_i)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:12<00:00,  5.82it/s]


In [35]:
# disp3(structures_i) # missing segment information, segment position information, pp position information

In [36]:
# somehow the sequence lengths and polypeptide lengths are of unequal lengths... HOW?!
"""for i in range(50):
    print(structures_i['prot_len'].iloc[i], structures_i['pp_lens'].iloc[i])"""

"for i in range(50):\n    print(structures_i['prot_len'].iloc[i], structures_i['pp_lens'].iloc[i])"

In [37]:
full_a = structure_to_full(active, structures_a, al_df)

In [38]:
full_a.to_csv('full_a.csv')
full_a = pd.read_csv('full_a.csv')

In [39]:
# full_a['full_prot_seq']

In [40]:
# full_a['TM7_combined']

In [41]:
# full_a['H8_combined']

In [42]:
complete_a = complete_data(full_a)

In [43]:
# disp3(complete_a)

In [44]:
# Filter out poorly aligned proteins
# disp3(complete_a[complete_a['std'] < 20])

In [45]:
#complete_i['pp_lens']

In [46]:
full_i = structure_to_full(inactive, structures_i, al_df)

In [47]:
full_i.to_csv('full_i.csv')
full_i = pd.read_csv('full_i.csv')

In [48]:
# print(len(full_i))

In [49]:
complete_i = complete_data(full_i)

In [50]:
# disp3(complete_i[complete_i['std'] < 10])

In [51]:
# disp3(complete_i['start'])

In [52]:
# len(complete_i[complete_i['std'] < 10])

In [53]:
# print(len(complete_i))

In [54]:
# Fix that number ~ we have 73 proteins in the full datatable but end up with 87 
# after the segments alignment is complete (unfiltered)
# disp3(complete_i['TM7_combined'])

In [67]:
disp3(complete_i[['PDB', 'TM7_combined', 'target_wrt_tm7']])

,PDB,TM7_combined,target_wrt_tm7
5,6PS1,KEVYILLNWIGYVNSGFNPLIYCR,"(17, 23, 19.6, 1.8547236990991407, 1.980000000..."
6,6PS6,KEVYILLNWIGYVNSGFNPLIYCR,"(17, 23, 19.6, 1.8547236990991407, 1.980000000..."
7,6PS5,KEVYILLNWIGYVNSGFNPLIYCR,"(17, 23, 19.6, 1.8547236990991407, 1.980000000..."


In [56]:
target = 'NPFIY'

In [122]:
def get_target_df(complete: pd.DataFrame, target='NPXXY', valid=True):
    def sum_start(a, b):
        return a + b[0]
    def sum_end(a, b):
        return a + b[1]
    def extend(ls):
        is_list = isinstance(ls[0], list)
        for i, l in enumerate(ls):
            if is_list & i == 0:
                out = l
            elif is_list:
                out.extend(l)
            else:
                return ls
        return out
    def clean_pps(x):
        return ast.literal_eval(x.replace('Seq', '').replace('(', '').replace(')', ''))
    target_df = complete[['PDB', 'uniprot(gene)', 'Resolution', 'PDB date', \
                          'TM7_combined', 'score', 'prot_len', 'full_prot_seq']].copy()
    target_df['pp_seqs'] = complete.apply(lambda x: clean_pps(x.pp_seqs), axis=1).copy()
    target_df['pp_seq_lens'] = target_df.apply(lambda x: [len(y) for y in x.pp_seqs], axis=1).copy()
    target_df['start'] = complete['target_wrt_tm7'].apply(lambda x: x[0]).copy()
    target_df['end'] = complete['target_wrt_tm7'].apply(lambda x: x[1]).copy()
    # target_df.loc[:, 'target_seq'] = target
    target_df['start_absolute'] = complete.apply(lambda x: sum_start(x.start, x.target_wrt_tm7), axis=1).copy()
    target_df['end_absolute'] = complete.apply(lambda x: sum_end(x.start, x.target_wrt_tm7), axis=1).copy()
    target_df['full_aligned_seg'] = target_df.apply(lambda x: x.full_prot_seq[x.start_absolute:x.end_absolute], axis=1)
    target_df['TM7_aligned_seg'] = target_df.apply(lambda x: x.TM7_combined[x.start:x.end], axis=1)
    target_df['psi_phi'] = complete.apply(lambda x: extend(ast.literal_eval(x.psi_phi)), axis=1)
    if valid:
        target_df['target_angles'] = target_df.apply(lambda x: x.psi_phi[x.start_absolute:x.end_absolute], axis=1)
        target_df = target_df[target_df['full_aligned_seg']==target_df['TM7_aligned_seg']]
        return target_df  # [len(target_df['target_angles']) >= len(target)]
    else:
        return target_df

In [123]:
valid_a = get_target_df(complete_a, target, valid=True)

In [124]:
valid_a.to_csv('valid_a.csv')

In [125]:
valid_i = get_target_df(complete_i, target, valid=True)
# Filter for proteins where the alignments match ~ indicates that angles are valid and the region is present
# if only valids we compare if the full aligned segment withthe TM7 aligned segment
valid_i.to_csv('valid_i.csv')

In [126]:
disp3(valid_a)

,PDB,uniprot(gene),Resolution,PDB date,TM7_combined,score,prot_len,full_prot_seq,pp_seqs,pp_seq_lens,start,end,start_absolute,end_absolute,full_aligned_seg,TM7_aligned_seg,psi_phi,target_angles
7,6E67,ADRB2,3.7,2019-06-05,RKEVYILLNWIGYVNSGFNP,0.72,477,FLLAPNRSHAPDHDVTQQRDEVWVVGMGIVMSLIVLAIVFGNVLVI...,[FLLAPNRSHAPDHDVTQQRDEVWVVGMGIVMSLIVLAIVFGNVLV...,[60],8,15,444,451,NWIGYVN,NWIGYVN,"[(None, 0.4238732643408313), (-1.3269517941706...","[(-1.2472881356459822, -0.7438190232059247), (..."


In [127]:
disp3(valid_i)

,PDB,uniprot(gene),Resolution,PDB date,TM7_combined,score,prot_len,full_prot_seq,pp_seqs,pp_seq_lens,start,end,start_absolute,end_absolute,full_aligned_seg,TM7_aligned_seg,psi_phi,target_angles
5,6PS1,ADRB2,3.2,2019-11-13,KEVYILLNWIGYVNSGFNPLIYCR,1.258333,450,DEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITS...,[DEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFIT...,[60],17,23,421,427,NPLIYC,NPLIYC,"[(None, 2.5256872329638655), (-1.0111750694833...","[(-0.8380310981643087, -0.8003636915192417), (..."
6,6PS6,ADRB2,2.7,2019-11-13,KEVYILLNWIGYVNSGFNPLIYCR,1.254167,470,RDEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFIT...,[RDEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFI...,[60],17,23,422,428,NPLIYC,NPLIYC,"[(None, 0.27689763066035566), (-0.793226952381...","[(-0.8287561614163858, -0.8590474805592928), (..."
7,6PS5,ADRB2,2.9,2019-11-13,KEVYILLNWIGYVNSGFNPLIYCR,1.258333,454,DEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITS...,[DEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFIT...,[60],17,23,421,427,NPLIYC,NPLIYC,"[(None, 2.9306733892826236), (-1.0267900603457...","[(-0.8621097350371153, -0.8237030852743529), (..."


In [128]:
valid_a.target_angles

7    [(-1.2472881356459822, -0.7438190232059247), (...
Name: target_angles, dtype: object

In [129]:
np.asarray(valid_i['target_angles'].iloc[0])

array([[-0.8380311 , -0.80036369],
       [-1.10427995, -0.60851755],
       [-1.21469441, -0.48920851],
       [-1.36171012, -0.44312874],
       [-1.24633608, -0.13742096],
       [-1.19782354, -0.27717333]])

In [131]:
np.asarray(valid_a['target_angles'].iloc[0])

array([[-1.24728814, -0.74381902],
       [-1.07233937, -0.55704763],
       [-1.04399194, -0.63983771],
       [-1.17025408, -0.5595981 ],
       [-1.07773467, -0.610312  ],
       [-1.16985303, -0.48814673],
       [-1.07865644, -0.54541424]])

In [130]:
valid_i.to_csv('valid_i.csv')

In [210]:
# j, o, u and x do not code for an amino acid (sometimes used to represent ?)

In [205]:
len(target_df_i[target_df_i['full_aligned_seg']==target_df_i['TM7_aligned_seg']])

60

In [207]:
x = target_df_i[['prot_len', 'psi_phi']]

In [208]:
x['len_psi_phi'] = target_df_i.apply(lambda x: len(x.psi_phi), axis=1)

<ipython-input-208-d74df7e050d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['len_psi_phi'] = target_df_i.apply(lambda x: len(x.psi_phi), axis=1)


In [209]:
x

,prot_len,psi_phi,len_psi_phi
0,263,"[(None, -0.21198449177483827), (-0.98627837835...",224
1,227,"[(None, 2.237376884279098), (-2.41453240735885...",220
2,120,"[(None, 0.7647017437707673), (-1.5994704005879...",120
3,214,"[(None, 2.4882132440314564), (-1.5096841734862...",214
4,214,"[(None, 2.476242607922282), (-1.51911109054375...",214
...,...,...,...
82,403,"[(None, 1.9029684965799571), (-1.0835479430442...",388
83,578,"[(None, 1.6566621979967036), (-1.0567447797281...",389
84,403,"[(None, 1.92407071659809), (-1.086512390112896...",388
85,554,"[(None, 1.7258592945865587), (-1.0383658683555...",388


In [ ]:
# match each of the active complexes to all the inactive ones (based on the uniprot genes)

In [85]:
# interaction dataframe

interaction_df = pd.DataFrame(columns=['PDB_active', 'PDB_inactive', 'phi_psi_active', 'phi_psi_inactive', 'delta_phi_psi'])
# this should be done here...